SATIVA requires a 'taxonomy file', i.e. a text file that links each sequence record to a ncbi taxonomic id (aka taxid - see here).

We are going to generate this file from a bunch of sequence records downloaded from Genbank.

We have downloaded our reference sequences from Genbank in the so-called, genbank format, which is a rich format containing all sorts of metadata for every sequence record, including a specific taxonomic id (taxid), i.e. a identity number that uniquely identifies every species (and also higher taxonomic levels, such as genera, families, classes, etc) recorded on Genbank.

Let's parse our Genbank file using Biopython.


In [1]:
from Bio import SeqIO

records = SeqIO.to_dict(SeqIO.parse(open('../1-fetch_clean_align/12S_preSATIVA_fish.gb','r'),'genbank'))

In the below cell we display the first of our sequence records in Genbank format. Try to identify the taxid of the record and use NCBI's taxonomy database (here) to retrieve the full taxonomic tree for the taxon.

In [2]:
for r in records.keys():
    print records[r].format('genbank')
    break

LOCUS       KC292935                1462 bp    DNA     linear   VRT 13-JUN-2013
DEFINITION  Cyprinus carpio isolate WKL42 tRNA-Pro gene, D-loop, and tRNA-Phe
            gene, complete sequence; and 12S ribosomal RNA gene, partial
            sequence; mitochondrial.
ACCESSION   KC292935
VERSION     KC292935.1
KEYWORDS    .
SOURCE      mitochondrion Cyprinus carpio (common carp)
  ORGANISM  Cyprinus carpio
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Actinopterygii; Neopterygii; Teleostei; Ostariophysi; Cypriniformes;
            Cyprinidae; Cyprinus.
REFERENCE   1  (bases 1 to 1462)
  AUTHORS   Hao,J., Yang,Q., Bao,D., Liang,A.-J., Zhang,X.-H. and Dong,S.
  TITLE     The sequence comparison of mtDNA D-loop and adjacent regions in six
            fish species
  JOURNAL   Dalian Hai Yang Da Xue Xue Bao 28 (2), 160-165 (2013)
REFERENCE   2  (bases 1 to 1462)
  AUTHORS   Hao,J., Dong,S., Yang,Q., Bao,D., Liang,A. and Zhang,X.
  TITLE     Direct

We don't want to make our life harder than it already is and deal with subspecies so in the tax file for SATIVA we'll only do species.

Iterate over all records and check if taxid is subspecies, if yes replace with taxid of species and extract taxonomy line. If it's a subspecies record the record id for further processing.


In [3]:
from collections import defaultdict

taxon_to_taxid = {}
recs_to_adjust = []
taxon_to_recs = defaultdict(list)

for key in records.keys():
    r = records[key]
   
    source = [f for f in r.features if f.type == 'source'][0]
    if (len(source.qualifiers['organism'][0].split(" ")) == 2):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
#            print source.qualifiers['db_xref']
            for t in source.qualifiers['db_xref']:
#                print t
                if 'taxon' in t:
                    if not source.qualifiers['organism'][0] in taxon_to_taxid:
                        print " .. add to records - %s" %t.split(":")[1]
                        taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                    else:
                        print " .. already covered"
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    elif (len(source.qualifiers['organism'][0].split(" ")) > 2) and ((source.qualifiers['organism'][0].split(" ")[1] == 'sp.') or ('cf.' in source.qualifiers['organism'][0].split(" "))):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
            for t in source.qualifiers['db_xref']:
                if 'taxon' in t:
                    if not str(t) in taxon_to_taxid:
                        taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                    else:
                        print " .. already covered" 
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    else:
        print "subspecies: %s" %source.qualifiers['organism'][0]
        recs_to_adjust.append(r.id)

Cyprinus carpio  .. add to records - 7962
Perca fluviatilis  .. add to records - 8168
Cobitis taenia  .. add to records - 98395
subspecies: Cyprinus carpio haematopterus
Pungitius pungitius  .. add to records - 134920
Solea solea  .. add to records - 90069
subspecies: Gasterosteus aculeatus williamsoni
Esox lucius  .. add to records - 8010
Cobitis taenia  .. already covered
Hypophthalmichthys molitrix  .. add to records - 13095
Oncorhynchus mykiss  .. add to records - 8022
Rhodeus sericeus  .. add to records - 58327
Hypophthalmichthys molitrix  .. already covered
Phoxinus phoxinus  .. add to records - 58324
Carassius auratus  .. add to records - 7957
Hypophthalmichthys molitrix  .. already covered
Salmo salar  .. add to records - 8030
Ambloplites rupestris  .. add to records - 109273
Coregonus lavaretus  .. add to records - 59291
Alburnus alburnus  .. add to records - 54556
Hypophthalmichthys molitrix  .. already covered
Salmo salar  .. already covered
Acipenser sturio  .. add to recor

In [4]:
print recs_to_adjust

['JX188254.1', 'KM273828.1', 'AP011236.1', 'HQ875340.1', 'KM282418.1', 'EU048341.1', 'KP171723.1', 'HQ391896.1', 'GU233801.1', 'AB111951.1', 'JN105357.1', 'AB379915.1', 'LC069402.1', 'KC894466.1', 'KC992395.1', 'AM910409.1', 'JX188253.1', 'GU086397.1', 'AP011239.1', 'GU086395.1', 'KF856964.1', 'KT634053.1']


For the records that were identified as being *subspecies* reduce to species and check whether the taxid for the species had already been encountered before. If not we'd need to fetch it from NCBI.

In [5]:
from collections import defaultdict

to_fetch = defaultdict(list)

for key in records.keys():
    r = records[key]
    if r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
        else:
            to_fetch[adjust_to].append(r.id)

Cyprinus carpio haematopterus -> Cyprinus carpio
Gasterosteus aculeatus williamsoni -> Gasterosteus aculeatus
Carassius auratus auratus -> Carassius auratus
Carassius auratus ssp. 'Pingxiang' -> Carassius auratus
Gasterosteus aculeatus williamsoni -> Gasterosteus aculeatus
Salmo trutta macrostigma -> Salmo trutta
Salmo trutta fario -> Salmo trutta
Micropterus salmoides salmoides -> Micropterus salmoides
Salmo trutta fario -> Salmo trutta
Carassius auratus auratus -> Carassius auratus
Cyprinus carpio 'wuyuanensis' -> Cyprinus carpio
Carassius auratus auratus -> Carassius auratus
Carassius auratus ssp. 'Kinbuna' -> Carassius auratus
Abramis brama orientalis -> Abramis brama
Phoxinus phoxinus tumensis -> Phoxinus phoxinus
Salmo trutta trutta -> Salmo trutta
Cyprinus carpio 'color' -> Cyprinus carpio
Carassius auratus auratus -> Carassius auratus
Carassius auratus grandoculis -> Carassius auratus
Carassius auratus auratus -> Carassius auratus
Cyprinus carpio 'wananensis' -> Cyprinus carpio

Check if we are good, or if there are any taxids missing.

In [6]:
if to_fetch:
    print "need to fetch some taxids"
    print to_fetch
else:
    print "Have taxids for all records"

Have taxids for all records


If there are any taxids missing the next cell will fetch them.

In [7]:
#from Bio import Entrez
#Entrez.email = "L.Harper@2015.hull.ac.uk"

#for binomial in to_fetch:
#    print binomial
#    handle = Entrez.esearch(db="Taxonomy", term=binomial)
#    record = Entrez.read(handle)
#    print record["IdList"][0]
#    taxon_to_taxid[binomial] = record["IdList"][0]
    
#    taxon_to_recs[binomial] = to_fetch[binomial]

Write taxids to file and fetch full taxonomy for all of them using taxit from the taxtastic package.

In [8]:
taxids = []

out=open("taxids.txt",'w')
for sp in taxon_to_taxid:
    taxids.append(taxon_to_taxid[sp])
    out.write(taxon_to_taxid[sp]+"\n")
out.close()

An IndexError may be returned if record names do not match the taxonomy e.g. Lixus cf. pulverulentus AS-2013 instead of Lixus pulverulentus.

If this occurs, you must go back to code cell 3 and change the code to split the record name where random letters occur (cf.)

Create comma-delimited text file with full taxonomic tree for each taxid: `taxa.csv`.

In [14]:
!taxit taxtable  /usr/bin/taxonomy.db -f taxids.txt -o taxa.csv

The metaBEAT image comes prepared with a database that contains taxonomic information for taxids. However, new taxids are constantly being added to Genbank, and the database (it's stored in the image in `/usr/bin/taxonomy.db`) contained in the image is not necessarily up to date. If the above command gives you errors about certain taxids not being found in the database you can try to update the database using code below. This will take a few minutes, depending on your internet connection.

In [10]:
%%bash

# Remove the old database
rm /usr/bin/taxonomy.db

# Create a new database. This command will first download the latest NCBI taxonomy info (taxdmp.zip) and configure the database
taxit new_database -d /usr/bin/taxonomy.db

# Remove temporal file that has been downloaded by the previous command
rm /usr/bin/taxdmp.zip

downloading ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdmp.zip to /usr/bin/taxdmp.zip
creating new database in /usr/bin/taxonomy.db using data in /usr/bin/taxdmp.zip
0 records lack primary names


Now, rerun code to create 'taxa.csv' file with full taxonomic tree for each taxid, using the latest taxonomy database.

In [11]:
!taxit taxtable /usr/bin/taxonomy.db -f taxids.txt -o taxa.csv

The full 'taxonomy string' for a given taxon as returned from NCBI could look, e.g. like so:

cellular organisms; Eukaryota; Opisthokonta; Metazoa; Eumetazoa; Bilateria; Deuterostomia; Chordata; Craniata; Vertebrata; Gnathostomata; Teleostomi; Euteleostomi; Actinopterygii; Actinopteri; Neopterygii; Teleostei; Osteoglossocephalai; Clupeocephala; Euteleosteomorpha; Protacanthopterygii; Salmoniformes; Salmonidae; Salmoninae; Salmo; Salmo trutta

The number of taxonomic levels in the taxonomy string may vary between taxa. Some taxonomic groups are classified into relatively uncommon intermediate taxonomic levels, that may not exist for other taxa.

In order to make our lives easier downstream we will limit ourselves only to a defined set of the most common taxonomic levels, that should be known for pretty much all taxa.

  -  superkingdom
  -  phylum
  -  class
  -  order
  -  family
  -  genus
  -  species

Extract 'taxonomy string' for a specific set of taxonomic levlels.


In [12]:
from collections import defaultdict

tax_levels=['superkingdom','phylum','class','order','family','genus','species']
indices = []
taxdict = defaultdict(list)
taxids_to_taxonomy = {}

infile=open("taxa.csv",'r')
header=infile.next()

header_as_list=header.strip().replace('"','').split(",")
for i in range(len(header_as_list)):
#    print header_as_list[i]
    if header_as_list[i] in tax_levels:
#        print "\t"+header_as_list[i],i
        indices.append(i)

for line in infile:
    line_as_list=line.strip().replace('"',"").split(",")
    taxdict[line_as_list[0]] = line_as_list[1:]

infile.close()

for t in taxids:
    print t,
#    print taxdict[t]
    taxonomy=""
    for i in range(len(tax_levels)):
        if taxdict[t][indices[i]-1] == "":
            taxonomy+='unknown'+';'
        else:
            taxonomy+=taxdict[taxdict[t][indices[i]-1]][2]+";"
    print taxonomy[:-1]
    taxids_to_taxonomy[t] = taxonomy[:-1]

48668 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Rutilus;Rutilus rutilus
8030 Eukaryota;Chordata;Actinopteri;Salmoniformes;Salmonidae;Salmo;Salmo salar
27704 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Gobio;Gobio gobio
58325 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Leuciscus;Leuciscus leuciscus
69944 Eukaryota;Chordata;Actinopteri;Gadiformes;Lotidae;Lota;Lota lota
278164 Eukaryota;Chordata;Actinopteri;Clupeiformes;Clupeidae;Alosa;Alosa alosa
54556 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Alburnus;Alburnus alburnus
36185 Eukaryota;Chordata;Actinopteri;Salmoniformes;Salmonidae;Thymallus;Thymallus thymallus
7748 Eukaryota;Chordata;unknown;Petromyzontiformes;Petromyzontidae;Lampetra;Lampetra fluviatilis
40830 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Barbus;Barbus barbus
8010 Eukaryota;Chordata;Actinopteri;Esociformes;Esocidae;Esox;Esox lucius
58317 Eukaryota;Chordata;Actinopteri;Cypriniformes;Cyprinidae;Blicca;Blicca bj

Write out *.tax file for SATIVA.

In [13]:
out=open("tax_for_SATIVA.tax", 'w')

for sp in taxon_to_recs:
    for rec in taxon_to_recs[sp]:
        out.write("%s\t%s\n" %(rec,taxids_to_taxonomy[taxon_to_taxid[sp]]))
        
out.close()